<a href="https://colab.research.google.com/github/RezaBahani/IUTComputationalPhysics/blob/main/CriticalTempratureUsingIsingModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm
import tqdm.contrib
import tqdm.contrib.concurrent
from scipy.ndimage import convolve
from concurrent.futures import ThreadPoolExecutor

In [3]:
class IsingModel(np.ndarray):
    def __new__(cls , N , *args , **kwargs ):
        obj = np.random.choice([1 , -1] , size=[N,N] ).view(cls)
        return obj

    def __array_finalize__(self , obj):
        if obj is None: return

    def __init__(self , N):
        self.N = N
        self._filter = np.array([[0, 1, 0], [1, 0, 1], [0, 1, 0]])
        self._neighbors = convolve(self , self.filter , mode='wrap')

    @property
    def magnetization(self):
        return float(np.sum(self))

    @property
    def filter(self):
        return self._filter

    @property
    def neighbors(self):
        return self._neighbors

    @property
    def energy(self):
        return float(-0.5*np.sum( self*self.neighbors ))

    def monte_carlo_step(self, temperature : float ) -> bool:
        """
            Perform a single Monte Carlo sweep with sequential spin updates.
            Returns the number of accepted spin flips.
        """
        i = np.random.randint(0, self.N-1)
        j = np.random.randint(0, self.N-1)

        # Calculate the energy change if this spin is flipped
        delta_E = 2 * self[i, j] * self.neighbors[i, j]
        # Apply Metropolis criterion
        if delta_E <= 0 or np.random.rand() < np.exp(-delta_E / temperature):
            self[i, j] *= -1  # Flip the spin
            # Update the n_sum matrix around the (i, j) element
            self._neighbors[ i+1 % self.N , j] += 2 * self[i, j]
            self._neighbors[ i-1 , j] += 2 * self[i, j]
            self._neighbors[ i , j+1 % self.N] += 2 * self[i, j]
            self._neighbors[ i , j-1] += 2 * self[i, j]
            return True
        return False

In [8]:
N_Size = 10

def simulate_ising_model(T, N = N_Size, steps_to_equilibruim = 2000, steps_to_average = 100000):

    E, M, A = np.zeros(steps_to_average), np.zeros(steps_to_average), np.zeros(steps_to_average)
    E_ = np.zeros(steps_to_equilibruim)
    model = IsingModel(N)

    for i in range(steps_to_equilibruim):
        model.monte_carlo_step(T)
        E_[i] = model.energy


    for i in range(steps_to_average):
        res = model.monte_carlo_step(T)
        E[i] = model.energy
        M[i] = model.magnetization
        A[i] = res

    return T, E_, E, M, A


temperature = np.linspace(0.1, 4.1, 10)
N_Simulation = 20

with ThreadPoolExecutor() as executor:
    results = list(tqdm.contrib.concurrent.thread_map(simulate_ising_model, np.repeat(temperature, N_Simulation)))

  0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
M_avg = []
M_stdev = []

allEs = np.concatenate([ r[2] for r in results ])
e_bins = np.linspace(np.min(allEs) , np.max(allEs) , 20)
for T in temperature:
    res = [ r for r in results if r[0] == T ]
    Es_ = [ r[1] for r in res ]
    Es = [ r[2] for r in res ]
    Ms = [ r[3] for r in res ]
    As = [ r[4] for r in res ]

    fig, axs = plt.subplots(2, 1, sharex=False, figsize=(10, 8))
    fig.suptitle(f"T={T:.2f}")
    boltzman = np.exp( -( e_bins[:-1] - e_bins[0] ) / T)
    def ehist(E, T):
        h , b = np.histogram(E , bins=e_bins)
        hf = h.astype(float )
        hf /= boltzman
        return hf
    for i, E in enumerate(Es):
        axs[0].bar( e_bins[:-1] , ehist(E , T) , width=i/len(Es) , label=f'energy distribution {i}' , alpha=0.5)
    axs[0].plot( e_bins[:-1] , ehist( np.concatenate(Es) , T )/N_Simulations , label='average')
    #axs[0].plot( e_bins[:-1] , boltzman , label='Boltzmann')
    axs[0].set_yscale('log')
    axs[0].set_ylabel("# of states")
    axs[0].set_xlabel("Energy")
    axs[0].set_xlim( np.min(allEs) , np.max(allEs) )
    #axs[0].legend()

    for i, E_ in enumerate(Es_):
        axs[1].plot(E_, label=f'energy over time {i}' , alpha=0.5)
    axs[1].set_ylabel("Energy")
    axs[1].set_xlabel("Step")
    #axs[1].legend()

    plt.show()
    M_vals = [ np.mean( np.abs( M[::1] ) ) for M in Ms ]
    M_avg.append( np.mean(M_vals) )
    M_stdev.append( np.std(M_vals) )

#now plot magnetization with error bars
plt.errorbar(temperatures, M_avg , yerr=M_stdev , fmt='o' , label='Magnetization')
plt.legend()
plt.title("Magnetization vs Temperature")
plt.xlabel("Temperature")
plt.ylabel("Magnetization")
plt.show()

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()